In [1]:
!pip install wandb pyyaml opencv-python;

In [2]:
import os
import yaml
import glob
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import wandb
from wandb.integration.keras import WandbCallback
import matplotlib.pyplot as plt

2025-05-22 15:23:44.227181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-22 15:23:44.244826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-22 15:23:44.250219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-22 15:23:44.264686: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-22 15:23:45.036643: W tensorflow/compiler/tf2

In [3]:
wandb.login(key="017feefe0af6702cda76aab121ec71cf3a362fec")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: samutheman (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
dataset_yaml = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml"
images_trainset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/train"
labels_trainset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train"
images_valset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val"
labels_valset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/val"
images_testset = False#"/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/test"
labels_testset = False#"/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/test"


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
with open(dataset_yaml, "r") as f:
    config = yaml.safe_load(f)


In [7]:
# --- Parameters (tweak these to your dataset) ---
IMG_SIZE    = 516
GRID_SIZE   = 16
NUM_CLASSES = 1   #
NUM_BOXES   = 3    # number of anchors per cell
OUTPUT_SHAPE = (GRID_SIZE, GRID_SIZE, NUM_BOXES * (5 + NUM_CLASSES))

In [8]:
def build_yolo_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    def conv_bn_leaky(x, filters):
        x = layers.Conv2D(filters, 3, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        return layers.LeakyReLU(alpha=0.1)(x)

    x = conv_bn_leaky(inputs, 16)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 32)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 64)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 128)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 256)
    x = layers.Dropout(0.3)(x)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 512)
    x = layers.Dropout(0.3)(x)

    # Final prediction conv
    x = layers.Conv2D(
        NUM_BOXES * (5 + NUM_CLASSES),
        kernel_size=1,
        padding='same',
        activation='sigmoid'  # or linear + custom loss
    )(x)

    # Check shape compatibility
    output = layers.Reshape((GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))(x)

    return tf.keras.Model(inputs, output)


In [9]:
# --- YOLO loss with binary class ---
def yolo_loss(y_true, y_pred):
    # reshape to (..., S, S, B, 5 + C)
    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    obj_mask = true[..., 4:5]

    # localization losses
    xy_loss = tf.reduce_sum(obj_mask * tf.square(true[..., 0:2] - pred[..., 0:2]))
    wh_loss = tf.reduce_sum(obj_mask * tf.square(
        tf.sqrt(true[..., 2:4] + 1e-6) - tf.sqrt(pred[..., 2:4] + 1e-6)
    ))

    # objectness and class losses
    obj_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(true[..., 4:5], pred[..., 4:5])
    )
    class_bce = tf.keras.losses.binary_crossentropy(true[..., 5:], pred[..., 5:])
    class_bce = tf.expand_dims(class_bce, axis=-1)
    class_loss = tf.reduce_sum(obj_mask * class_bce)

    return xy_loss + wh_loss + obj_loss + class_loss

In [10]:
def area_difference_loss(y_true, y_pred):
    # Reshape to (..., S, S, B, 5 + C)
    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))

    # Calculate area = w * h * objectness
    pred_area = pred[..., 2] * pred[..., 3] * pred[..., 4]
    true_area = true[..., 2] * true[..., 3] * true[..., 4]

    # Sum all areas in the batch
    total_pred_area = tf.reduce_sum(pred_area)
    total_true_area = tf.reduce_sum(true_area)

    # Loss is the absolute difference in total area
    return tf.abs(total_pred_area - total_true_area)

In [27]:
def learned_weight_loss(y_true, y_pred, model):
    yolo = yolo_loss(y_true, y_pred)
    area = area_difference_loss(y_true, y_pred)

    yolo = yolo / (tf.reduce_mean(yolo) + 1e-6)
    area = area / (tf.reduce_mean(area) + 1e-6)

    ls_yolo = model.log_sigma_yolo
    ls_area = model.log_sigma_area

    total = (
        tf.exp(-2.0 * ls_yolo) * yolo +
        tf.exp(-2.0 * ls_area) * area +
        ls_yolo + ls_area
    )
    tf.summary.scalar("loss/yolo_raw", tf.reduce_mean(yolo))
    tf.summary.scalar("loss/area_raw", tf.reduce_mean(area))
    tf.summary.scalar("loss/yolo_weighted", tf.reduce_mean(tf.exp(-2.0 * ls_yolo) * yolo))
    tf.summary.scalar("loss/area_weighted", tf.reduce_mean(tf.exp(-2.0 * ls_area) * area))
    tf.summary.scalar("loss/total", tf.reduce_mean(total))

    return tf.reduce_mean(total)

In [12]:
def build_yolo_model_with_uncertainty():
    # your existing YOLO backbone
    model = build_yolo_model()

    # add two trainable log-variance weights to the model so the optimiser sees them
    model.log_sigma_yolo = model.add_weight(
        name="log_sigma_yolo", shape=(), initializer="zeros", trainable=True
    )
    model.log_sigma_area = model.add_weight(
        name="log_sigma_area", shape=(), initializer="zeros", trainable=True
    )
    return model


In [15]:
class SigmaLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({
            "log_sigma_yolo": self.model.log_sigma_yolo.numpy(),
            "log_sigma_area": self.model.log_sigma_area.numpy(),
        })


In [16]:
def combined_yolo_area_loss(y_true, y_pred, area_weight=0.1):
    # Standard YOLO loss
    yolo = yolo_loss(y_true, y_pred)
    # Area difference loss
    area = area_difference_loss(y_true, y_pred)
    # Combine with weighting
    return yolo + area_weight * area

In [17]:
def apply_nms(pred_grid, conf_thresh=0.3, iou_thresh=0.4):
    boxes = []
    scores = []

    for gy in range(GRID_SIZE):
        for gx in range(GRID_SIZE):
            for b in range(NUM_BOXES):
                pred = pred_grid[gy, gx, b]
                conf = pred[4]
                if conf < conf_thresh:
                    continue

                x, y, w, h = pred[:4]
                cx = (gx + x) / GRID_SIZE
                cy = (gy + y) / GRID_SIZE
                xmin = cx - w / 2
                ymin = cy - h / 2
                xmax = cx + w / 2
                ymax = cy + h / 2

                boxes.append([ymin, xmin, ymax, xmax])
                scores.append(conf)

    if not boxes:
        return [], []

    boxes = tf.constant(boxes, dtype=tf.float32)
    scores = tf.constant(scores, dtype=tf.float32)

    selected_indices = tf.image.non_max_suppression(
        boxes,
        scores,
        max_output_size=10,
        iou_threshold=iou_thresh,
        score_threshold=conf_thresh
    )

    selected_boxes = tf.gather(boxes, selected_indices).numpy()
    selected_scores = tf.gather(scores, selected_indices).numpy()

    return selected_boxes, selected_scores



In [18]:
def build_yolo_label_grid(label_path, grid_size, num_boxes, num_classes):
    label_grid = np.zeros((grid_size, grid_size, num_boxes, 5 + num_classes), dtype=np.float32)

    if not os.path.exists(label_path):
        return label_grid

    with open(label_path, "r") as f:
        for line in f:
            class_id, x, y, w, h = map(float, line.strip().split())
            gx = min(int(x * grid_size), grid_size - 1)
            gy = min(int(y * grid_size), grid_size - 1)
            cell_x = x * grid_size - gx
            cell_y = y * grid_size - gy

            anchor_idx = 0

            label_grid[gy, gx, anchor_idx, 0:5] = [cell_x, cell_y, w, h, 1.0]
            label_grid[gy, gx, anchor_idx, 5 + int(class_id)] = 1.0

    return label_grid

def prepare_image_label_paths(image_dir, label_dir, grid_size, num_boxes, num_classes):
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
    label_grids = []

    for img_path in image_paths:
        filename = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, f"{filename}.txt")
        label_grid = build_yolo_label_grid(label_path, grid_size, num_boxes, num_classes)
        label_grids.append(label_grid)

    return image_paths, np.array(label_grids, dtype=np.float32)


In [19]:
def parse_image_and_label(image_path, label_tensor, img_size):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0

    # Explicit shape setting is critical here
    img.set_shape([img_size, img_size, 3])
    label_tensor.set_shape([GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES])

    return img, label_tensor


def make_yolo_dataset(image_paths, label_tensors, img_size, batch_size=4, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((image_paths, label_tensors))
    
    if shuffle:
        ds = ds.shuffle(buffer_size=len(image_paths))
    
    def process(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_size, img_size])
        img = img / 255.0

        img.set_shape([img_size, img_size, 3])
        label.set_shape([GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES])

        return img, label

    ds = ds.map(process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [20]:
train_image_paths, train_labels = prepare_image_label_paths(
    images_trainset, labels_trainset, GRID_SIZE, NUM_BOXES, NUM_CLASSES
)

val_image_paths, val_labels = prepare_image_label_paths(
    images_valset, labels_valset, GRID_SIZE, NUM_BOXES, NUM_CLASSES
)


In [21]:
sweep_config = {
    "method": "random",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"values": [1e-4, 5e-4, 1e-3]},
        "batch_size":    {"values": [4, 8]}
    }
}


In [22]:
def sweep_train():
    wandb.init(project="Own CNN", name=None, config=config)
    cfg = wandb.config

    run_count = getattr(wandb.run, "sweep_iteration", None)
    wandb.run.name = f"CNNModelUnc_{run_count if run_count is not None else wandb.run.id}"

    # Build model with the extra trainable weights
    model = build_yolo_model_with_uncertainty()

    # Wrap the loss so it can “see” the model’s log-sigma variables
    def loss_fn(y_true, y_pred):
        return learned_weight_loss(y_true, y_pred, model)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.learning_rate),
        loss=loss_fn
    )

    train_ds = make_yolo_dataset(train_image_paths, train_labels,
                                 IMG_SIZE, batch_size=cfg.batch_size)
    val_ds   = make_yolo_dataset(val_image_paths,   val_labels,
                                 IMG_SIZE, batch_size=cfg.batch_size, shuffle=False)

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,
        callbacks=[WandbCallback(save_graph=False, save_model=False),  SigmaLogger()]
    )


In [28]:
sweep_id = wandb.sweep(sweep_config, project="Own CNN")
wandb.agent(sweep_id, function=sweep_train, count=10)

Create sweep with ID: hiijpgq0
Sweep URL: https://wandb.ai/rueedi-tobias-hochschule-luzern/Own%20CNN/sweeps/hiijpgq0


wandb: Agent Starting Run: n627s0ns with config:
wandb: 	batch_size: 4
wandb: 	learning_rate: 0.0001
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: 
wandb: 🚀 View run pretty-sweep-1 at: https://wandb.ai/rueedi-tobias-hochschule-luzern/Own%20CNN/runs/4ojs1ziq
wandb: Find logs at: wandb/run-20250522_152708-4ojs1ziq/logs


/opt/conda/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 124s 40ms/step - loss: 1.9009 - val_loss: 1.7308
Epoch 2/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 98s 36ms/step - loss: 1.7143 - val_loss: 1.6937
Epoch 3/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6933 - val_loss: 1.6931
Epoch 4/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6931 - val_loss: 1.6931
Epoch 5/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6931 - val_loss: 1.6931
Epoch 6/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6931 - val_loss: 1.6931
Epoch 7/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6931 - val_loss: 1.6931
Epoch 8/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6931 - val_loss: 1.6931
Epoch 9/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 99s 36ms/step - loss: 1.6931 - val_loss: 1.6931
Epoch 10/10
2715/2715 ━━━━━━━━━━━━━━━━━━━━ 108s 40ms/step - loss: 1.6931 - val_loss: 1.6931


...

/opt/conda/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/10
2194/2715 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - loss: 1.7580

wandb: Ctrl + C detected. Stopping sweep.


2270/2715 ━━━━━━━━━━━━━━━━━━━━ 16s 37ms/step - loss: 1.7564

wandb: ERROR Problem finishing run
Exception in thread Thread-19 (_run_job):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_68286/2595073259.py", line 25, in sweep_train
  File "/opt/conda/lib/python3.12/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/lib/python3.12/site-packages/ipykernel/iostream.py", line 694, in write
    self._schedule_flush()
  File "/opt/conda/lib/python3.12/site-packages/ipykernel/iostream.py", line 590, in _schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "/opt/conda/lib/python3.12/site-packages/ipykernel/iostream.py", line 267, in schedule
    self._event_pipe.send(b"")
  File "/opt/conda/lib/python3.12/site-packages/zmq/sugar/socket.py", line 698, in send
    return super().send(data, flags=flags, copy=copy, trac

In [24]:
def draw_nms_boxes(image, label_grid, pred_boxes):
    img = (image * 255).astype(np.uint8).copy()
    h, w, _ = img.shape
    cell_h, cell_w = h // GRID_SIZE, w // GRID_SIZE

    # Draw ground truth boxes (green)
    for gy in range(GRID_SIZE):
        for gx in range(GRID_SIZE):
            for b in range(NUM_BOXES):
                if label_grid[gy, gx, b, 4] > 0.5:
                    x, y, bw, bh = label_grid[gy, gx, b, :4]
                    cx = int((gx + x) * cell_w)
                    cy = int((gy + y) * cell_h)
                    bw = int(bw * w)
                    bh = int(bh * h)
                    cv2.rectangle(img, (cx - bw // 2, cy - bh // 2), (cx + bw // 2, cy + bh // 2), (0, 255, 0), 2)

    # Draw NMS-filtered prediction boxes (red)
    for box in pred_boxes:
        ymin, xmin, ymax, xmax = box
        x1 = int(xmin * w)
        y1 = int(ymin * h)
        x2 = int(xmax * w)
        y2 = int(ymax * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return img



In [ ]:
for i, (image_tensor, label_tensor) in enumerate(val_ds.take(10)):
    image_batch = image_tensor.numpy()  # (B, H, W, 3)
    label_batch = label_tensor.numpy()  # (B, GRID, GRID, NUM_BOXES, 5 + C)

    for b in range(image_batch.shape[0]):
        img_np = image_batch[b]
        label_np = label_batch[b]
        
        # Run prediction on a single image
        pred = model.predict(img_np[np.newaxis])[0]  # shape: (GRID, GRID, NUM_BOXES, 5 + C)

        # --- Apply NMS ---
        nms_boxes, scores = apply_nms(pred, conf_thresh=0.3, iou_thresh=0.4)

        # --- Draw boxes ---
        vis_img = draw_nms_boxes(img_np, label_np, nms_boxes)  # custom function below

        plt.imshow(vis_img)
        plt.axis('off')
        plt.title(f"Green = GT, Red = Pred (sample {i}-{b})")
        plt.show()
